In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "micls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9132 | AUPR: 0.6221 | Acc: 95.11%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9144 | AUPR: 0.6274 | Acc: 95.08%
model(kernel=linear, C=1)
Valid:         AUC: 0.9130 | AUPR: 0.6225 | Acc: 95.07%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9144 | AUPR: 0.6274 | Acc: 95.08%
Internal test: AUC: 0.9228 | AUPR: 0.6687 | Acc: 94.84%
External test: AUC: 0.9140 | AUPR: 0.6134 | Acc: 94.63%


In [3]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7575 | AUPR: 0.4220 | Acc: 94.62%
model(n_neighbors=5)
Valid:         AUC: 0.7870 | AUPR: 0.4715 | Acc: 94.94%
model(n_neighbors=7)
Valid:         AUC: 0.8039 | AUPR: 0.4955 | Acc: 95.03%
best model(n_neighbors=7)
Valid:         AUC: 0.8039 | AUPR: 0.4955 | Acc: 95.03%
Internal test: AUC: 0.7965 | AUPR: 0.5010 | Acc: 94.53%
External test: AUC: 0.8028 | AUPR: 0.4904 | Acc: 94.71%


In [4]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8736 | AUPR: 0.4085 | Acc: 86.82%
best model()
Valid:         AUC: 0.8736 | AUPR: 0.4085 | Acc: 86.82%
Internal test: AUC: 0.8830 | AUPR: 0.4387 | Acc: 86.74%
External test: AUC: 0.8710 | AUPR: 0.3900 | Acc: 85.28%


In [5]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9158 | AUPR: 0.6511 | Acc: 95.32%
model(C=10.0)
Valid:         AUC: 0.9155 | AUPR: 0.6519 | Acc: 95.33%


/home/yan/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


model(C=100.0)
Valid:         AUC: 0.9152 | AUPR: 0.6515 | Acc: 95.32%
best model(C=1.0)
Valid:         AUC: 0.9158 | AUPR: 0.6511 | Acc: 95.32%
Internal test: AUC: 0.9250 | AUPR: 0.6914 | Acc: 95.18%
External test: AUC: 0.9146 | AUPR: 0.6396 | Acc: 94.93%


/home/yan/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [6]:
from sklearn.neural_network import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=1000,
    activation='relu',
    solver='sgd', nesterovs_momentum=True, momentum=0.9)
param_grid_list = [{
    # 'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
    'hidden_layer_sizes': [(50, 50)],
    'learning_rate_init': [0.0003, 0.001],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(50, 50), learning_rate_init=0.0003)
Valid:         AUC: 0.9022 | AUPR: 0.6179 | Acc: 95.15%
model(hidden_layer_sizes=(50, 50), learning_rate_init=0.001)
Valid:         AUC: 0.9133 | AUPR: 0.6479 | Acc: 95.33%
best model(hidden_layer_sizes=(50, 50), learning_rate_init=0.001)
Valid:         AUC: 0.9133 | AUPR: 0.6479 | Acc: 95.33%
Internal test: AUC: 0.9241 | AUPR: 0.6747 | Acc: 95.16%
External test: AUC: 0.9143 | AUPR: 0.6355 | Acc: 94.84%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8756 | AUPR: 0.5590 | Acc: 86.22%
model(criterion=log_loss)
Valid:         AUC: 0.8726 | AUPR: 0.5550 | Acc: 84.14%
model(criterion=entropy)
Valid:         AUC: 0.8726 | AUPR: 0.5550 | Acc: 84.14%
best model(criterion=gini)
Valid:         AUC: 0.8756 | AUPR: 0.5590 | Acc: 86.22%
Internal test: AUC: 0.8915 | AUPR: 0.6281 | Acc: 83.80%
External test: AUC: 0.8765 | AUPR: 0.5759 | Acc: 83.14%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9184 | AUPR: 0.6590 | Acc: 89.74%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9195 | AUPR: 0.6598 | Acc: 90.12%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9199 | AUPR: 0.6610 | Acc: 90.24%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9173 | AUPR: 0.6580 | Acc: 89.86%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9187 | AUPR: 0.6615 | Acc: 90.25%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9193 | AUPR: 0.6631 | Acc: 90.31%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9173 | AUPR: 0.6580 | Acc: 89.86%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9187 | AUPR: 0.6615 | Acc: 90.25%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9193 | AUPR: 0.6631 | Acc: 90.31%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9199 | AUPR: 0.6610 | Acc: 9

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9194 | AUPR: 0.6422 | Acc: 95.09%
model(criterion=squared_error)
Valid:         AUC: 0.9200 | AUPR: 0.6412 | Acc: 95.11%
best model(criterion=squared_error)
Valid:         AUC: 0.9200 | AUPR: 0.6412 | Acc: 95.11%
Internal test: AUC: 0.9277 | AUPR: 0.6789 | Acc: 95.40%
External test: AUC: 0.9200 | AUPR: 0.6398 | Acc: 95.02%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9152 | AUPR: 0.6447 | Acc: 95.32%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9156 | AUPR: 0.6465 | Acc: 95.32%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6501 | Acc: 95.32%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6501 | Acc: 95.32%
Internal test: AUC: 0.9217 | AUPR: 0.6845 | Acc: 95.38%
External test: AUC: 0.9122 | AUPR: 0.6357 | Acc: 94.91%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)